<a href="https://colab.research.google.com/github/Suchitra-V31/Natural_Language_processing/blob/main/Whatsapp_Chat_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install emoji --upgrade

     |████████████████████████████████| 184 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=6cd2c93fcb62e136be7bd8afecc80272f1b2fec0fd5e8cabdc02d3712bc9b851
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji


Let us import the necessary libraries

In [6]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import emoji
from PIL import Image
from collections import Counter
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator

In [15]:
#Extract time
def date_time(s):
  pattern='^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
  result=re.match(pattern,s)  #re.match() function of re in Python will search the regular expression pattern and return the first occurrence
  if result:
    return True
  return False
#Find Authors or contacts
def find_author(s):
  s=s.split(':')
  if len(s)==2:
    return True
  return False
def getDataPoint(line):
  splitline=line.split('-')
  datetime=splitline[0]
  date,time=datetime.split(',')
  message=' '.join(splitline[1:])
  if find_author(message):
    splitmessage=message.split(':')
    author=splitmessage[0]
    message=' '.join(splitmessage[1:])
  else:
    author=None
  return date,time,author,message


In [17]:
 data=[]
conversation='/content/WhatsApp Chat with Kirthiga.txt'
with open(conversation,encoding='utf-8') as fp:
  fp.readline()
  messageBuffer=[]
  date,time,author=None,None,None
  while True:
    line=fp.readline()
    if not line:
      break
    line=line.strip()
    if date_time(line):
      if len(messageBuffer)>0:
        data.append([date,time,author,' '.join(messageBuffer)])
      messageBuffer.clear()
      date,time,author,message =getDataPoint(line)
      messageBuffer.append(message)
    else:
      messageBuffer.append(line)

In the previous code we have split the data into time date author(contact name) and our mesage...let us create a dataframe with these features.

In [18]:
df=pd.DataFrame(data,columns=['date','time','author','message'])
df['date']=pd.to_datetime(df['date'])

In [19]:
df.head()

,date,time,author,message
0,2019-08-13,7:10 PM,Kirthiga,
1,2019-08-13,8:12 PM,Suji,Hi
2,2019-08-13,8:12 PM,Suji,Un Peru sollu d
3,2019-08-13,8:14 PM,Kirthiga,Hii
4,2019-08-13,8:14 PM,Kirthiga,Kirthiga


In [37]:
#let us drop all the nan values
data=df.dropna()

**VADER ( Valence Aware Dictionary for Sentiment Reasoning)** is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the NLTK package and can be applied directly to unlabeled text data.

In [38]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [39]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [40]:
sentiments=SentimentIntensityAnalyzer()

In [42]:
data['Positive']=[sentiments.polarity_scores(i)['pos'] for i in data['message']]
data['Negative']=[sentiments.polarity_scores(i)['neg'] for i in data['message']]
data['Neutral']=[sentiments.polarity_scores(i)['neu'] for i in data['message']]

In [43]:
data.head()

,date,time,author,message,Positive,Negative,Neutral
0,2019-08-13,7:10 PM,Kirthiga,,0.0,0.0,0.0
1,2019-08-13,8:12 PM,Suji,Hi,0.0,0.0,1.0
2,2019-08-13,8:12 PM,Suji,Un Peru sollu d,0.0,0.0,1.0
3,2019-08-13,8:14 PM,Kirthiga,Hii,0.0,0.0,1.0
4,2019-08-13,8:14 PM,Kirthiga,Kirthiga,0.0,0.0,1.0


In [44]:
x=sum(data['Positive'])
y=sum(data['Negative'])
z=sum(data['Neutral'])

In [46]:
x,y,z

(563.6560000000001, 181.21399999999994, 14887.127999999946)

In [55]:
def sentiment_score(a,b,c):
  if a>b and a>c:
    print("Positive")
    print(emoji.emojize(":grinning_face_with_big_eyes:"))
  elif b>c and b>a:
    print("Negative")
    print(emoji.emojize(':upside-down_face:'))
  else:
    print("Neutral")
    print(emoji.emojize(':grinning_squinting_face:'))

In [56]:
sentiment_score(x,y,z)

Neutral
😆
